In [3]:
import sklearn
sklearn.__version__

'0.24.2'

- kserve/sklearnserver 의 package version 과 notebook 의 package version 을 맞춰준다. 

```
!pip install -r requirements.txt

scikit-learn==1.0.1
numpy==1.21.6
pandas==1.3.5
azure-storage-blob==12.9.0
kserve==0.9.0
```

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(100, 1) - 100, axis=0)
y = np.array([np.pi * np.sin(X).ravel(), np.pi * np.cos(X).ravel()]).T
y[::5, :] += 0.5 - rng.rand(20, 2)

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_3 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)

# Predict
X_test = np.arange(-100.0, 100.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)
y_3 = regr_3.predict(X_test)


In [9]:
y_3

array([[ 1.82569761,  2.64490159],
       [ 1.82569761,  2.64490159],
       [ 1.82569761,  2.64490159],
       ...,
       [-1.84319053, -0.55258289],
       [-1.84319053, -0.55258289],
       [-1.84319053, -0.55258289]])

In [ ]:
# https://github.com/kserve/kserve/blob/master/python/sklearnserver/sklearnserver/test_sklearn_model_repository.py
# model save 에 사용할수 있는 lib

In [14]:
import joblib

In [18]:
import os
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

bucket_name="kserve-test"
model_path="model.joblib"
joblib.dump(regr_3, model_path)

['model.joblib']

In [19]:
# !mc alias set kfminio http://minio-service.kubeflow:9000 minio minio123
# !mc config host ls
# !mc cp vol-1/complete/model.joblib kfminio/kserve-test/sklearn-test/model.joblib

```
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  annotations:
    sidecar.istio.io/inject: "false"
  name: "sklearn-custom"
spec:
  predictor:
    serviceAccountName: 'sa'
    sklearn:
      image: "kserve/sklearnserver:v0.9.0"
      storageUri: "s3://kserve-test/sklearn-test"
```

In [21]:
from kserve import (
    KServeClient,
    utils
)
from kubernetes import client 

In [24]:
namespace = utils.get_default_target_namespace()
service_name = "sklearn-custom"
KServe = KServeClient()
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

NAME                 READY                           PREV                    LATEST URL                                                              
sklearn-custom       True                               0                       100 http://sklearn-custom.kubeflow-user-example-com.example.com      


In [25]:
isvc_resp

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2022-08-30T16:28:04Z',
  'finalizers': ['inferenceservice.finalizers'],
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:serviceAccountName': {},
       'f:sklearn': {'.': {}, 'f:image': {}, 'f:storageUri': {}}}}},
    'manager': 'Swagger-Codegen',
    'operation': 'Update',
    'time': '2022-08-30T16:28:01Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:finalizers': {}},
     'f:spec': {'f:predictor': {'f:containers': {},
       'f:sklearn': {'f:args': {}}}},
     'f:status': {'.': {},
      'f:address': {'.': {}, 'f:url': {}},
  

In [38]:
X_test_list = X_test.tolist()

In [39]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

http://sklearn-custom.kubeflow-user-example-com.svc.cluster.local/v1/models/sklearn-custom:predict


In [40]:
sklearn_test_input={
    "instances": X_test_list
}

In [41]:
response = requests.post(isvc_url, json=sklearn_test_input)
print(response.text)

{"predictions": [[1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.8256976133184155, 2.6449015931558], [1.82569

In [46]:
response.json()["predictions"] == y_3

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       ...,
       [ True,  True],
       [ True,  True],
       [ True,  True]])

In [1]:
# https://github.com/kserve/kserve/blob/master/python/sklearnserver/sklearnserver/model.py